In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
!nvidia-smi

Tue Mar 23 23:42:05 2021       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 460.56       Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla P100-PCIE...  Off  | 00000000:00:04.0 Off |                    0 |
| N/A   62C    P0    48W / 250W |      0MiB / 16280MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [ ]:
%%capture
!pip install datasets==1.4.1
!pip install transformers==4.4.0
!pip install torchaudio
!pip install librosa
!pip install jiwer

In [ ]:
import re
import json
import random
import torchaudio
import librosa
import torch
import numpy as np
import pandas as pd
import numpy as np
import IPython.display as ipd
from dataclasses import dataclass, field
from typing import Any, Dict, List, Optional, Union
from IPython.display import display, HTML
from datasets import ClassLabel, load_dataset, load_metric
from transformers import (
    Wav2Vec2CTCTokenizer, 
    Wav2Vec2FeatureExtractor, 
    Wav2Vec2Processor, 
    Wav2Vec2ForCTC,
    TrainingArguments,
    Trainer
)

LANG_CODE = "ky"
SAVE_PATH = f"/content/drive/MyDrive/wav2vec2-large-xlsr-{LANG_CODE}"
CHARS_TO_IGNORE_REGEX = '[\,\?\.\!\-\;\:\"\“\%\‘\”\�\–\—\¬\⅛]'

## Load Data

In [ ]:
common_voice_train = load_dataset("common_voice", LANG_CODE, split="train+validation")
common_voice_test = load_dataset("common_voice", LANG_CODE, split="test")

Couldn't find file locally at common_voice/common_voice.py, or remotely at https://raw.githubusercontent.com/huggingface/datasets/1.4.1/datasets/common_voice/common_voice.py.
The file was picked from the master branch on github instead at https://raw.githubusercontent.com/huggingface/datasets/master/datasets/common_voice/common_voice.py.
Reusing dataset common_voice (/root/.cache/huggingface/datasets/common_voice/ky/6.1.0/0041e06ab061b91d0a23234a2221e87970a19cf3a81b20901474cffffeb7869f)
Couldn't find file locally at common_voice/common_voice.py, or remotely at https://raw.githubusercontent.com/huggingface/datasets/1.4.1/datasets/common_voice/common_voice.py.
The file was picked from the master branch on github instead at https://raw.githubusercontent.com/huggingface/datasets/master/datasets/common_voice/common_voice.py.
Reusing dataset common_voice (/root/.cache/huggingface/datasets/common_voice/ky/6.1.0/0041e06ab061b91d0a23234a2221e87970a19cf3a81b20901474cffffeb7869f)


In [ ]:
common_voice_train = common_voice_train.remove_columns(["accent", "age", "client_id", "down_votes", "gender", "locale", "segment", "up_votes"])
common_voice_test = common_voice_test.remove_columns(["accent", "age", "client_id", "down_votes", "gender", "locale", "segment", "up_votes"])

## Prepare Vocab

In [ ]:
def remove_special_characters(batch):
    batch["sentence"] = re.sub(CHARS_TO_IGNORE_REGEX, '', batch["sentence"]).lower() + " "
    return batch

def show_random_elements(dataset, num_examples=10):
    assert num_examples <= len(dataset), "Can't pick more elements than there are in the dataset."
    picks = []
    for _ in range(num_examples):
        pick = random.randint(0, len(dataset)-1)
        while pick in picks:
            pick = random.randint(0, len(dataset)-1)
        picks.append(pick)
    
    df = pd.DataFrame(dataset[picks])
    display(HTML(df.to_html()))

common_voice_train = common_voice_train.map(remove_special_characters)
common_voice_test = common_voice_test.map(remove_special_characters)
show_random_elements(common_voice_train.remove_columns(["path"]))

Loading cached processed dataset at /root/.cache/huggingface/datasets/common_voice/ky/6.1.0/0041e06ab061b91d0a23234a2221e87970a19cf3a81b20901474cffffeb7869f/cache-3e5ac03cdec29865.arrow
Loading cached processed dataset at /root/.cache/huggingface/datasets/common_voice/ky/6.1.0/0041e06ab061b91d0a23234a2221e87970a19cf3a81b20901474cffffeb7869f/cache-7425638f03578aca.arrow


,sentence
0,шаардык милиция адам өлтүрүү беренеси боюнча кылмыш ишин козгогон
1,көзөмөл бар болгону менен конкреттүү коррупция болуп атат
2,саясатчылар сегизбаевдин сотко таасир этип жатканына ишенишет
3,мен мечитте бул тема жөнүндө сүйлөп атканыма уялып жатам
4,кээ бирлери кыргызстандагы жакындарын сагынганын айтышты
5,православ чиркөөсү калаанын борбордук бөлүгүнөн орун алган
6,бир жылдын ичинде он мыйзам жазыптырмын
7,милициянын кызматкерин өлтүрүү фактысы боюнча кылмыш иши козголду
8,биз базардын токтогул көчөсү тарабынан чыгып келе жатканбыз
9,түндөсү саат тогуздарда алар мени үйүнө алып кетишти


In [ ]:
def build_vocab_dict():
    vocab_train = extract_chars_from_dataset(common_voice_train)
    vocab_test = extract_chars_from_dataset(common_voice_test)
    vocab_list = list(set(vocab_train["vocab"][0]) | set(vocab_test["vocab"][0]))
    vocab_dict= {v: k for k, v in enumerate(vocab_list)}
    vocab_dict["|"] = vocab_dict[" "]
    del vocab_dict[" "]
    vocab_dict["[UNK]"] = len(vocab_dict)
    vocab_dict["[PAD]"] = len(vocab_dict)
    return vocab_dict


def extract_chars_from_dataset(dataset):
    return dataset.map(
        extract_all_chars, 
        batched=True, 
        batch_size=-1, 
        keep_in_memory=True, 
        remove_columns=common_voice_train.column_names
)


def extract_all_chars(batch):
  all_text = " ".join(batch["sentence"])
  vocab = list(set(all_text))
  return {"vocab": [vocab], "all_text": [all_text]}


def save_vocab(vocab_dict):
    with open(f'{SAVE_PATH}/vocab.json', 'w') as vocab_file:
        json.dump(vocab_dict, vocab_file)


vocab_dict = build_vocab_dict()
save_vocab(vocab_dict)
print(len(vocab_dict))
vocab_dict


40


{'[PAD]': 39,
 '[UNK]': 38,
 '|': 13,
 'а': 28,
 'б': 8,
 'в': 10,
 'г': 27,
 'д': 19,
 'е': 33,
 'ж': 6,
 'з': 5,
 'и': 35,
 'й': 24,
 'к': 14,
 'л': 16,
 'м': 21,
 'н': 29,
 'о': 22,
 'п': 3,
 'р': 30,
 'с': 23,
 'т': 18,
 'у': 12,
 'ф': 9,
 'х': 1,
 'ц': 7,
 'ч': 25,
 'ш': 32,
 'щ': 37,
 'ъ': 36,
 'ы': 0,
 'ь': 4,
 'э': 2,
 'ю': 11,
 'я': 17,
 'ё': 20,
 'ң': 31,
 'ү': 26,
 'ӊ': 15,
 'ө': 34}

## Load Processor

In [ ]:
tokenizer = Wav2Vec2CTCTokenizer(f"{SAVE_PATH}/vocab.json", unk_token="[UNK]", pad_token="[PAD]", word_delimiter_token="|")
feature_extractor = Wav2Vec2FeatureExtractor(feature_size=1, sampling_rate=16000, padding_value=0.0, do_normalize=True, return_attention_mask=True)
processor = Wav2Vec2Processor(feature_extractor=feature_extractor, tokenizer=tokenizer)
processor.save_pretrained(SAVE_PATH)

## Preprocess Datasets

In [ ]:
def process_dataset(dataset):
    dataset = dataset.map(
        speech_file_to_array_fn, 
        remove_columns=dataset.column_names
    )
    dataset = dataset.map(resample, num_proc=4)
    dataset = dataset.map(
        prepare_dataset, 
        remove_columns=dataset.column_names, 
        batch_size=8, 
        num_proc=4, 
        batched=True
    )
    return dataset


def speech_file_to_array_fn(batch):
    speech_array, sampling_rate = torchaudio.load(batch["path"])
    batch["speech"] = speech_array[0].numpy()
    batch["sampling_rate"] = sampling_rate
    batch["target_text"] = batch["sentence"]
    return batch


def resample(batch):
    batch["speech"] = librosa.resample(np.asarray(batch["speech"]), 48_000, 16_000)
    batch["sampling_rate"] = 16_000
    return batch


def prepare_dataset(batch):
    # check that all files have the correct sampling rate
    assert (
        len(set(batch["sampling_rate"])) == 1
    ), "Make sure all inputs have the same sampling rate."

    batch["input_values"] = processor(
        batch["speech"], 
        sampling_rate=batch["sampling_rate"][0]
    ).input_values
  
    with processor.as_target_processor():
        batch["labels"] = processor(batch["target_text"]).input_ids 
            
    return batch

common_voice_train = process_dataset(common_voice_train)
common_voice_test = process_dataset(common_voice_test)

Loading cached processed dataset at /root/.cache/huggingface/datasets/common_voice/ky/6.1.0/0041e06ab061b91d0a23234a2221e87970a19cf3a81b20901474cffffeb7869f/cache-fa64b8b6eb1d6372.arrow


Loading cached processed dataset at /root/.cache/huggingface/datasets/common_voice/ky/6.1.0/0041e06ab061b91d0a23234a2221e87970a19cf3a81b20901474cffffeb7869f/cache-b442b23e20b96642.arrow


Loading cached processed dataset at /root/.cache/huggingface/datasets/common_voice/ky/6.1.0/0041e06ab061b91d0a23234a2221e87970a19cf3a81b20901474cffffeb7869f/cache-e510ddd3909f73ab.arrow


Loading cached processed dataset at /root/.cache/huggingface/datasets/common_voice/ky/6.1.0/0041e06ab061b91d0a23234a2221e87970a19cf3a81b20901474cffffeb7869f/cache-a8deebee2e1e15d7.arrow


Loading cached processed dataset at /root/.cache/huggingface/datasets/common_voice/ky/6.1.0/0041e06ab061b91d0a23234a2221e87970a19cf3a81b20901474cffffeb7869f/cache-81a9465072f04456.arrow
/usr/local/lib/python3.7/dist-packages/numpy/core/_asarray.py:83: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  return array(a, dtype, copy=False, order=order)


Loading cached processed dataset at /root/.cache/huggingface/datasets/common_voice/ky/6.1.0/0041e06ab061b91d0a23234a2221e87970a19cf3a81b20901474cffffeb7869f/cache-a567284c8899dd81.arrow


Loading cached processed dataset at /root/.cache/huggingface/datasets/common_voice/ky/6.1.0/0041e06ab061b91d0a23234a2221e87970a19cf3a81b20901474cffffeb7869f/cache-68871a5f124c3628.arrow


Loading cached processed dataset at /root/.cache/huggingface/datasets/common_voice/ky/6.1.0/0041e06ab061b91d0a23234a2221e87970a19cf3a81b20901474cffffeb7869f/cache-f9d52f788fadff16.arrow
Loading cached processed dataset at /root/.cache/huggingface/datasets/common_voice/ky/6.1.0/0041e06ab061b91d0a23234a2221e87970a19cf3a81b20901474cffffeb7869f/cache-2ec1b3b9677418ef.arrow
Loading cached processed dataset at /root/.cache/huggingface/datasets/common_voice/ky/6.1.0/0041e06ab061b91d0a23234a2221e87970a19cf3a81b20901474cffffeb7869f/cache-b27d09cfed8fffc9.arrow


## Define Data Collator

In [ ]:
@dataclass
class DataCollatorCTCWithPadding:
    """
    Data collator that will dynamically pad the inputs received.
    Args:
        processor (:class:`~transformers.Wav2Vec2Processor`)
            The processor used for proccessing the data.
        padding (:obj:`bool`, :obj:`str` or :class:`~transformers.tokenization_utils_base.PaddingStrategy`, `optional`, defaults to :obj:`True`):
            Select a strategy to pad the returned sequences (according to the model's padding side and padding index)
            among:
            * :obj:`True` or :obj:`'longest'`: Pad to the longest sequence in the batch (or no padding if only a single
              sequence if provided).
            * :obj:`'max_length'`: Pad to a maximum length specified with the argument :obj:`max_length` or to the
              maximum acceptable input length for the model if that argument is not provided.
            * :obj:`False` or :obj:`'do_not_pad'` (default): No padding (i.e., can output a batch with sequences of
              different lengths).
        max_length (:obj:`int`, `optional`):
            Maximum length of the ``input_values`` of the returned list and optionally padding length (see above).
        max_length_labels (:obj:`int`, `optional`):
            Maximum length of the ``labels`` returned list and optionally padding length (see above).
        pad_to_multiple_of (:obj:`int`, `optional`):
            If set will pad the sequence to a multiple of the provided value.
            This is especially useful to enable the use of Tensor Cores on NVIDIA hardware with compute capability >=
            7.5 (Volta).
    """

    processor: Wav2Vec2Processor
    padding: Union[bool, str] = True
    max_length: Optional[int] = None
    max_length_labels: Optional[int] = None
    pad_to_multiple_of: Optional[int] = None
    pad_to_multiple_of_labels: Optional[int] = None

    def __call__(self, features: List[Dict[str, Union[List[int], torch.Tensor]]]) -> Dict[str, torch.Tensor]:
        # split inputs and labels since they have to be of different lenghts and need
        # different padding methods
        input_features = [{"input_values": feature["input_values"]} for feature in features]
        label_features = [{"input_ids": feature["labels"]} for feature in features]

        batch = self.processor.pad(
            input_features,
            padding=self.padding,
            max_length=self.max_length,
            pad_to_multiple_of=self.pad_to_multiple_of,
            return_tensors="pt",
        )
        with self.processor.as_target_processor():
            labels_batch = self.processor.pad(
                label_features,
                padding=self.padding,
                max_length=self.max_length_labels,
                pad_to_multiple_of=self.pad_to_multiple_of_labels,
                return_tensors="pt",
            )

        # replace padding with -100 to ignore loss correctly
        labels = labels_batch["input_ids"].masked_fill(labels_batch.attention_mask.ne(1), -100)

        batch["labels"] = labels

        return batch

data_collator = DataCollatorCTCWithPadding(processor=processor, padding=True)

## Prepare Evaluation Metric

In [ ]:
wer_metric = load_metric("wer")

def compute_metrics(pred):
    pred_logits = pred.predictions
    pred_ids = np.argmax(pred_logits, axis=-1)

    pred.label_ids[pred.label_ids == -100] = processor.tokenizer.pad_token_id

    pred_str = processor.batch_decode(pred_ids)
    # we do not want to group tokens when computing the metrics
    label_str = processor.batch_decode(pred.label_ids, group_tokens=False)

    wer = wer_metric.compute(predictions=pred_str, references=label_str)

    return {"wer": wer}

## Prepare Model

In [ ]:
model = Wav2Vec2ForCTC.from_pretrained(
    "facebook/wav2vec2-large-xlsr-53", 
    attention_dropout=0.1,
    hidden_dropout=0.1,
    feat_proj_dropout=0.0,
    mask_time_prob=0.05,
    layerdrop=0.1,
    gradient_checkpointing=True, 
    ctc_loss_reduction="mean", 
    pad_token_id=processor.tokenizer.pad_token_id,
    vocab_size=len(processor.tokenizer)
)

model.freeze_feature_extractor()

Some weights of Wav2Vec2ForCTC were not initialized from the model checkpoint at facebook/wav2vec2-large-xlsr-53 and are newly initialized: ['lm_head.bias', 'lm_head.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


## Train

In [ ]:
training_args = TrainingArguments(
  output_dir=SAVE_PATH,
  group_by_length=True,
  per_device_train_batch_size=16,
  gradient_accumulation_steps=2,
  evaluation_strategy="steps",
  num_train_epochs=50,
  fp16=True,
  save_steps=2000,
  eval_steps=400,
  logging_steps=400,
  learning_rate=3e-4,
  warmup_steps=500,
  save_total_limit=1,
)

trainer = Trainer(
    model=model,
    data_collator=data_collator,
    args=training_args,
    compute_metrics=compute_metrics,
    train_dataset=common_voice_train,
    eval_dataset=common_voice_test,
    tokenizer=processor.feature_extractor,
)

In [ ]:
trainer.train()

/usr/local/lib/python3.7/dist-packages/torch/optim/lr_scheduler.py:134: UserWarning: Detected call of `lr_scheduler.step()` before `optimizer.step()`. In PyTorch 1.1.0 and later, you should call them in the opposite order: `optimizer.step()` before `lr_scheduler.step()`.  Failure to do this will result in PyTorch skipping the first value of the learning rate schedule. See more details at https://pytorch.org/docs/stable/optim.html#how-to-adjust-learning-rate
  "https://pytorch.org/docs/stable/optim.html#how-to-adjust-learning-rate", UserWarning)


Step,Training Loss,Validation Loss,Wer,Runtime,Samples Per Second
400,5.218900,3.069143,1.000000,181.136000,8.298000
800,1.122100,0.504544,0.592792,182.723900,8.226000
1200,0.232000,0.491232,0.542160,184.241800,8.158000
1600,0.136000,0.515902,0.531290,183.621400,8.185000
2000,0.110200,0.544674,0.522965,183.827200,8.176000
2400,0.085900,0.574774,0.528450,184.636600,8.140000
2800,0.071000,0.586662,0.511115,183.311000,8.199000
3200,0.062000,0.584384,0.497992,182.941700,8.216000
3600,0.053300,0.563688,0.493194,183.018600,8.212000
4000,0.048100,0.565379,0.483498,183.998400,8.169000


TrainOutput(global_step=5400, training_loss=0.5391183931739242, metrics={'train_runtime': 34480.1315, 'train_samples_per_second': 0.157, 'total_flos': 2.6153617117992284e+19, 'epoch': 50.0, 'init_mem_cpu_alloc_delta': 356165, 'init_mem_gpu_alloc_delta': 1261919232, 'init_mem_cpu_peaked_delta': 18306, 'init_mem_gpu_peaked_delta': 0, 'train_mem_cpu_alloc_delta': 1585718, 'train_mem_gpu_alloc_delta': 3783649792, 'train_mem_cpu_peaked_delta': 157387260, 'train_mem_gpu_peaked_delta': 7369542144})

In [ ]:
trainer.save_model()

In [ ]:
trainer.evaluate()

{'epoch': 50.0,
 'eval_loss': 0.5650733709335327,
 'eval_mem_cpu_alloc_delta': 118943976,
 'eval_mem_cpu_peaked_delta': 12242433,
 'eval_mem_gpu_alloc_delta': -196608,
 'eval_mem_gpu_peaked_delta': 1631516160,
 'eval_runtime': 180.5374,
 'eval_samples_per_second': 8.325,
 'eval_wer': 0.47125648810106746}